In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from glob import glob
import os.path as osp
import panel as pn
import subprocess
import datetime
import os
from sfim_lib.io.afni import load_netcc
from sfim_lib.plotting.fc_matrices import hvplot_fc
from tqdm import tqdm
from utils.basics import PRCS_DATA_DIR, ATLASES_DIR, ATLAS_NAME, PRJ_DIR, CODE_DIR


/data/SFIMJGC_HCP7T/Apps/envs/bold_waves_2024a/lib/python3.9/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.4 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.4, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [2]:
import getpass
username = getpass.getuser()
print(username)

javiergc


In [3]:
import os
port_tunnel = int(os.environ['PORT2'])
print('++ INFO: Second Port available: %d' % port_tunnel)

++ INFO: Second Port available: 44177


In [4]:
ATLAS_NAME = 'Power264'
ATLAS_DIR = osp.join(ATLASES_DIR,ATLAS_NAME)

# 1. Load Dataset Information

In [5]:
dataset_info_df = pd.read_csv(osp.join(PRJ_DIR,'resources','good_scans.txt'))
dataset_info_df = dataset_info_df.set_index(['Subject','Session'])
print('++ Number of scans: %s scans' % dataset_info_df.shape[0])

++ Number of scans: 434 scans


# 2. Create Swarm Script to Extract ROI TS from fully denoised data

In [6]:
script_path = osp.join(PRJ_DIR,f'swarm.{username}',f'N03d_Post_AfniProc.{ATLAS_NAME}.sh')
print(script_path)

/data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc/N03d_Post_AfniProc.Power264.sh


In [7]:
log_path = osp.join(PRJ_DIR,f'logs.{username}',f'N03d_Post_AfniProc.{ATLAS_NAME}.log')
if not osp.exists(log_path):
    os.makedirs(log_path)
print(log_path)

/data/SFIMJGC_HCP7T/BCBL2024/logs.javiergc/N03d_Post_AfniProc.Power264.log


In [8]:
with open(script_path, 'w') as the_file:
    the_file.write('# Script Creation Date: %s\n' % str(datetime.date.today()))
    the_file.write(f'# swarm -f {script_path} -g 16 -t 8 -b 10 --time 00:20:00 --logdir {log_path} --partition quick,norm --module afni\n')
    the_file.write('\n')
    for sbj,ses in list(dataset_info_df.index):
        atlas_path  = f'{ATLASES_DIR}/{ATLAS_NAME}/{ATLAS_NAME}.nii.gz' 
        the_file.write(f'export SBJ={sbj} SES={ses} ATLAS_NAME={ATLAS_NAME} ATLAS_PATH={atlas_path} ATLASES_DIR={ATLASES_DIR}; sh  {CODE_DIR}/bash/N03d_Post_AfniProc.sh \n')
the_file.close()     

In [9]:
script_path

'/data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc/N03d_Post_AfniProc.Power264.sh'

```bash
cd /data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc
swarm -f /data/SFIMJGC_HCP7T/BCBL2024/swarm.javiergc/N03d_Post_AfniProc.Power264.sh -g 16 -t 8 -b 10 --time 00:20:00 --logdir /data/SFIMJGC_HCP7T/BCBL2024/logs.javiergc/N03d_Post_AfniProc.Power264.log --partition quick,norm --module afni
```

# 3. Check all expected datasets were processed

In [11]:
for sbj,ses in list(dataset_info_df.index):
    for e in ['e01','e02','e03']:
        netcc_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'errts.{sbj}.r01.{e}.meica_dn.scale.tproject_ALL.{ATLAS_NAME}_000.netts')
        if not osp.exists(netcc_path):
            print('++ WARNING: %s is missing'% netcc_path)

***

In [47]:
roi_info_path = osp.join(ATLAS_DIR,f'{ATLAS_NAME}.roi_info.csv')
roi_info_df   = pd.read_csv(roi_info_path)
roi_info_df

,ROI_ID,ROI_Name,pos_A,pos_R,pos_S,Network,Hemisphere,RGB
0,0,ROI000,-25,-98,-12,Uncertain,LH,#ffffff
1,1,ROI001,24,32,-18,Uncertain,RH,#ffffff
2,2,ROI002,-37,-29,-26,Uncertain,LH,#ffffff
3,3,ROI003,-7,-52,61,Sensory/somatomotor Hand,LH,#E0FFFF
4,4,ROI004,-14,-18,40,Sensory/somatomotor Hand,LH,#E0FFFF
...,...,...,...,...,...,...,...,...
229,229,ROI229,-27,-71,37,Dorsal attention,LH,#00ff00
230,230,ROI230,-32,-1,54,Dorsal attention,LH,#00ff00
231,231,ROI231,-42,-60,-9,Dorsal attention,LH,#00ff00
232,232,ROI232,-17,-59,64,Dorsal attention,LH,#00ff00


In [69]:
scenarios = ['E01-volreg','E02-volreg','E03-volreg','E01-MEICA','E02-MEICA','E03-MEICA']
fcs = {scenario:xr.DataArray(dims=['scan','roi_x','roi_y'], coords={'scan':['.'.join([sbj,ses]) for sbj,ses in dataset_info_df.index],
                                                                           'roi_x':list(roi_info_df['ROI_Name']),
                                                                           'roi_y':list(roi_info_df['ROI_Name'])}) for scenario in scenarios}

In [76]:
for sbj,ses in tqdm(list(dataset_info_df.index)):
    for scenario in scenarios:
        e = (scenario.split('-')[0]).lower()
        if 'volreg' in scenario:
            netcc_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'pb03.{sbj}.{e}.volreg.{ATLAS_NAME}_000.netcc')
        if 'MEICA' in scenario:
            netcc_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'errts.{sbj}.r01.{e}-MEICA.tproject.{ATLAS_NAME}_000.netcc')
        netcc = load_netcc(netcc_path)
        fcs[scenario].loc['.'.join([sbj,ses]),:,:] = netcc.values

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 436/436 [00:44<00:00,  9.89it/s]


In [77]:
mot_df = pd.DataFrame(index=np.arange(201),columns=list(dataset_info_df.index))
for sbj,ses in tqdm(list(dataset_info_df.index)):
    mot_path = osp.join(PRCS_DATA_DIR,sbj,f'D02_Preproc_fMRI_{ses}',f'motion_{sbj}_enorm.1D')
    mot_df[(sbj,ses)] = np.loadtxt(mot_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 436/436 [00:00<00:00, 814.18it/s]


In [78]:
scan_select = pn.widgets.Select(name='scan', options=list(fcs['E02-volreg'].coords['scan'].values))
@pn.depends(scan_select)
def plot_fc(scan):
    plots = pn.layout.GridBox(ncols=3)
    for scenario in scenarios:
        aux = fcs[scenario].sel(scan=scan).values
        aux = pd.DataFrame(aux,index=roi_info_df.set_index(['ROI_Name','ROI_ID','Hemisphere','Network','RGB']).index,
                               columns=roi_info_df.set_index(['ROI_Name','ROI_ID','Hemisphere','Network','RGB']).index)
        aux_plot = hvplot_fc(aux,major_label_overrides='regular_grid',cmap='RdBu_r', by='Network', add_labels=False, colorbar_position='left', net_cmap=nw_color_dict)
        plots.append(aux_plot)
    return pn.Row(plots)

In [79]:
@pn.depends(scan_select)
def plot_mot(scan):
    sbj,ses = scan.split('.')
    aux_df = pd.DataFrame(mot_df[(sbj,ses)].values,columns=['Motion [enorm]'])
    aux_df.index.name = 'TR'
    aux_df.name = 'Motion'
    return aux_df.hvplot()

In [80]:
dashboard = pn.Row(scan_select,pn.Column(plot_fc,plot_mot)).show(port=port_tunnel)

Launching server at http://localhost:44177


In [68]:
dashboard.stop()

In [62]:
import holoviews as hv

In [63]:
hv.help(pn.layout.FlexBox)

Parameters of 'FlexBox'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name                 Value           Type         Bounds     Mode  

align               'start'         Align                    V RW  
aspect_ratio          None          Aspect                 V RW AN 
css_classes            []            List       (0, None)    V RW  
design                None      ObjectSelector               V RW  
height                None         Integer      (0, None)  V RW AN 
min_width             None         Integer      (0, None)  V RW AN 
min_height            None         Integer      (0, None)  V RW AN 
max_width             None         Integer      (0, None)  V RW AN 
max_height            None         Integer      (0, None)  V RW AN 
margin                 0            Margin                 V RW AN 
styles                 {}            Dict                